In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [3]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [4]:
train_df = train_df.dropna()

In [5]:
train_cols = train_df.columns.tolist()
print(train_cols)

['Unnamed: 0', 'index', 'loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_

In [6]:
# Check Data Types
train_df.dtypes

Unnamed: 0                      int64
index                           int64
loan_amnt                     float64
int_rate                      float64
installment                   float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [7]:
train_df["loan_status"].value_counts()

high_risk    6090
low_risk     6090
Name: loan_status, dtype: int64

In [8]:
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [9]:
test_cols = test_df.columns.tolist()
print(test_cols)

['Unnamed: 0', 'index', 'loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_

In [10]:
compare = [i for i in test_cols if i not in train_cols]
compare

[]

In [11]:
test_df.dtypes

Unnamed: 0                      int64
index                           int64
loan_amnt                     float64
int_rate                      float64
installment                   float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [12]:
# Convert categorical data to numeric and separate target feature for testing data
train_df = pd.get_dummies(train_df, columns=["home_ownership","initial_list_status", 
                                             "application_type", "hardship_flag",
                                             "debt_settlement_flag", "verification_status", 
                                             "pymnt_plan"], drop_first=True)
train_df

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,initial_list_status_w,application_type_Joint App,hardship_flag_Y,debt_settlement_flag_Y,verification_status_Source Verified,verification_status_Verified
0,57107,57107,13375.0,0.1797,483.34,223000.0,low_risk,29.99,0.0,0.0,...,170200.0,1,0,0,1,0,0,0,0,0
1,141451,141451,21000.0,0.1308,478.68,123000.0,low_risk,11.26,2.0,0.0,...,35398.0,1,0,0,1,0,0,0,1,0
2,321143,321143,20000.0,0.1240,448.95,197000.0,low_risk,11.28,0.0,0.0,...,90340.0,1,0,0,1,0,0,0,1,0
3,11778,11778,3000.0,0.1240,100.22,45000.0,low_risk,18.08,0.0,0.0,...,15406.0,0,0,1,1,0,0,0,0,0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,low_risk,27.77,0.0,2.0,...,58778.0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,28000.0,high_risk,28.42,0.0,0.0,...,19055.0,0,0,1,1,0,0,0,0,0
12176,354944,354944,15000.0,0.1774,540.34,50000.0,high_risk,23.43,4.0,0.0,...,54824.0,0,0,1,1,0,0,0,0,1
12177,354973,354973,3600.0,0.1862,131.28,60000.0,high_risk,28.80,0.0,1.0,...,53065.0,0,0,1,1,0,0,0,0,0
12178,355002,355002,15000.0,0.0881,475.68,62000.0,high_risk,11.44,0.0,0.0,...,32930.0,1,0,0,1,1,0,0,1,0


In [13]:
test_df = pd.get_dummies(test_df, columns=["home_ownership","initial_list_status", 
                                             "application_type", "hardship_flag",
                                             "debt_settlement_flag", "verification_status", 
                                             "pymnt_plan"], drop_first=True)
test_df

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,initial_list_status_w,application_type_Joint App,hardship_flag_Y,verification_status_Source Verified,verification_status_Verified
0,67991,67991,40000.0,0.0819,814.70,140000.0,low_risk,19.75,0.0,1.0,...,74600.0,99475.0,1,0,0,1,0,0,0,0
1,25429,25429,6000.0,0.1524,208.70,55000.0,low_risk,11.52,2.0,0.0,...,5900.0,23628.0,0,0,1,1,0,0,0,0
2,38496,38496,3600.0,0.1695,128.27,42000.0,low_risk,6.74,0.0,0.0,...,7300.0,15000.0,0,0,1,1,0,0,0,0
3,19667,19667,20000.0,0.1524,478.33,100000.0,low_risk,12.13,0.0,2.0,...,13800.0,35981.0,0,0,1,1,0,0,0,0
4,37505,37505,3600.0,0.1240,120.27,50000.0,low_risk,16.08,0.0,3.0,...,21000.0,24977.0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,77282,77282,30000.0,0.1240,673.42,140480.0,high_risk,15.74,0.0,0.0,...,48400.0,107388.0,0,0,1,0,0,0,1,0
4698,77291,77291,24000.0,0.0756,747.22,50000.0,high_risk,26.81,0.0,0.0,...,13300.0,30775.0,0,0,1,1,0,0,0,0
4699,77292,77292,10000.0,0.2305,387.36,33000.0,high_risk,38.51,0.0,2.0,...,8500.0,29550.0,0,0,1,0,0,0,0,1
4700,77297,77297,8000.0,0.1862,205.86,38000.0,high_risk,16.36,0.0,1.0,...,1500.0,9657.0,0,0,1,1,0,0,1,0


In [14]:
X_train = train_df.drop("loan_status", axis=1)
y_train = train_df['loan_status']
X_train = pd.get_dummies(X_train)
X_train.drop(['Unnamed: 0','index'],axis=1,inplace=True)
X_train.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,initial_list_status_w,application_type_Joint App,hardship_flag_Y,debt_settlement_flag_Y,verification_status_Source Verified,verification_status_Verified
0,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,39728.0,...,170200.0,1,0,0,1,0,0,0,0,0
1,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,9585.0,...,35398.0,1,0,0,1,0,0,0,1,0
2,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,16708.0,...,90340.0,1,0,0,1,0,0,0,1,0
3,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,8809.0,...,15406.0,0,0,1,1,0,0,0,0,0
4,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,65420.0,...,58778.0,1,0,0,1,0,0,0,1,0


In [15]:
# add missing dummy variables to testing set
X_test = test_df.drop("loan_status", axis=1)
y_test = test_df['loan_status']
X_test = pd.get_dummies(X_test)
X_test.drop(['Unnamed: 0','index'],axis=1,inplace=True)
X_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,initial_list_status_w,application_type_Joint App,hardship_flag_Y,verification_status_Source Verified,verification_status_Verified
0,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,74600.0,99475.0,1,0,0,1,0,0,0,0
1,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,5900.0,23628.0,0,0,1,1,0,0,0,0
2,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,7300.0,15000.0,0,0,1,1,0,0,0,0
3,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,13800.0,35981.0,0,0,1,1,0,0,0,0
4,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,21000.0,24977.0,0,0,1,1,0,0,0,0


In [16]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [17]:
# Train a Random Forest Classifier model and print the model score
classifier.fit(X_train, y_train)

/Users/abayomi/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=1000)

In [25]:
Xtrain_cols = X_train.columns.tolist()
Xtest_cols = X_test.columns.tolist()

# Find missing column
missing_cols = [i for i in Xtrain_cols if i not in Xtest_cols]
missing_cols

['debt_settlement_flag_Y']

In [37]:
X_test["debt_settlement_flag_Y"] = 0

In [38]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [40]:
# Train the Logistic Regression model on the scaled data and print the model score
trainacc_log_unscaled=classifier.score(X_train, y_train)
testacc_log_unscaled=classifier.score(X_test, y_test)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7140394088669951
Testing Data Score: 0.5886856656741812


In [ ]:
# Train a Random Forest Classifier model on the scaled data and print the model score